In [1]:
!rm -rf /kaggle/working/*

In [2]:
# Install required packages
!pip install transformers
!pip install pyvi
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn
!pip install torch
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing install

In [3]:
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyvi import ViTokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score, 
    balanced_accuracy_score,
    f1_score,
    roc_auc_score,
    roc_curve, 
    precision_recall_curve,
    auc,
    average_precision_score,
    cohen_kappa_score,
    matthews_corrcoef,
    log_loss,
    hamming_loss,
    jaccard_score,
    top_k_accuracy_score
)
from sklearn.preprocessing import label_binarize
from sklearn.utils import resample
import logging
from datetime import datetime
import unicodedata
import zipfile
from IPython.display import FileLink, display

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

# Configuration

In [4]:
# Set up logging
log_dir = '/kaggle/working/'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'training_summary_sentiment_log_v2.txt')

In [5]:
# Remove existing log file if it exists
if os.path.exists(log_file):
    try:
        os.remove(log_file)
    except Exception as e:
        print(f"Could not remove existing log file: {e}")

# Configure logging
try:
    logging.getLogger().handlers.clear()
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.WARNING)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    logging.basicConfig(
        level=logging.INFO,
        handlers=[file_handler, console_handler],
        force=True
    )
    logging.info("Logging initialized successfully")
    print(f"Log file will be saved to: {log_file}")
except Exception as e:
    print(f"Failed to initialize logging to file: {e}. Using console logging only.")
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[console_handler]
    )

Log file will be saved to: /kaggle/working/training_summary_sentiment_log_v2.txt


In [6]:
# Seed everything for reproducibility
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configuration
EPOCHS = 10
BATCH_SIZE = 16
MAX_LEN = 256 
N_SPLITS = 5

# Data Processing

In [7]:
# Phase 1: Load Data
def load_original_data(xlsx_path):
    original_df = pd.read_excel(xlsx_path)
    df_processed = original_df[['summary', 'sentiment']].copy()
    logging.info(f"Original data loaded with shape: {original_df.shape}")
    assert original_df.equals(pd.read_excel(xlsx_path)), "Original data modified!"
    return df_processed

# Phase 2: Data Cleaning
def clean_data(df):
    # Check for NaN
    nan_rows = df[df['summary'].isna() | df['sentiment'].isna()]
    if not nan_rows.empty:
        df = df.dropna(subset=['summary', 'sentiment'])
        logging.info(f"Removed {len(nan_rows)} rows with NaN values")

    # Remove duplicate summaries
    initial_rows = len(df)
    df = df.drop_duplicates(subset=['summary'])
    logging.info(f"Removed {initial_rows - len(df)} duplicate summaries")

    # Remove short summaries
    df = df[df['summary'].str.len() >= 5]
    logging.info(f"Removed {initial_rows - len(df)} summaries shorter than 5 characters")

    # Ensure consistent sentiment labels
    valid_labels = {'Positive', 'Negative', 'Neutral'}
    invalid_labels = set(df['sentiment']) - valid_labels
    if invalid_labels:
        raise ValueError(f"Invalid sentiment labels found: {invalid_labels}")
    df['sentiment'] = df['sentiment'].str.strip()
    logging.info("Sentiment labels checked and stripped")
    return df

# Phase 3: Text Normalization
def normalize_text(text):
    text = unicodedata.normalize('NFC', str(text))
    text = ' '.join(text.split())
    return text

# Phase 4: Label Processing and Balancing
def process_labels(df):
    label2id = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
    id2label = {v: k for k, v in label2id.items()}
    df['sentiment'] = df['sentiment'].map(label2id)

    # Check class distribution
    distribution = df['sentiment'].value_counts().sort_index()
    logging.info(f"Class distribution before balancing: {distribution.to_dict()}")
    if distribution.min() / distribution.max() < 0.5:
        logging.warning("Significant class imbalance detected!")

    # Balance dataset using undersampling
    min_count = distribution.min()
    balanced_df = pd.concat([resample(df[df['sentiment'] == label], 
                           replace=False, n_samples=min_count, random_state=86)
                           for label in range(3)])
    balanced_df = balanced_df.sample(frac=1, random_state=86).reset_index(drop=True)
    logging.info(f"Class distribution after balancing: {balanced_df['sentiment'].value_counts().sort_index().to_dict()}")
    return balanced_df, label2id, id2label

# Phase 5: Tokenization
def tokenize_data(df, tokenizer):
    df['text'] = df['summary'].apply(normalize_text)
    df['tokenized'] = df['text'].apply(lambda x: tokenizer.encode_plus(
        x, max_length=MAX_LEN, padding='max_length', truncation=True,
        return_tensors='pt', add_special_tokens=True)['input_ids'].squeeze().numpy())
    logging.info(f"Tokenization completed with max_length={MAX_LEN}")
    return df


# Model Components

In [8]:
# Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['text']
        label = row['sentiment']

        encoding = self.tokenizer.encode_plus(
            text, max_length=self.max_len, padding='max_length',
            truncation=True, return_tensors='pt', add_special_tokens=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long)
        }

In [9]:
# Model Definition
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes=3):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        output = self.drop(pooled_output)
        return self.fc(output)

# Training Utilities

In [10]:
# Training and Evaluation Functions
def train_epoch(model, data_loader, criterion, optimizer, scheduler, device):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, targets)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return np.mean(losses), correct_predictions.double() / len(data_loader.dataset)

def eval_model(model, data_loader, criterion, device):
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)
            _, preds = torch.max(outputs, dim=1)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return np.mean(losses), correct_predictions.double() / len(data_loader.dataset)

def prepare_loaders(train_df, val_df, test_df, tokenizer):
    train_dataset = SentimentDataset(train_df, tokenizer)
    val_dataset = SentimentDataset(val_df, tokenizer)
    test_dataset = SentimentDataset(test_df, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, val_loader, test_loader

In [11]:
# Enhanced Evaluation Function with all metrics
def evaluate_model(model, data_loader, id2label):
    model.eval()
    all_preds = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)
            
            outputs = model(input_ids, attention_mask)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Use numeric labels instead of strings for metrics calculation
    classes = list(id2label.values())
    class_ids = list(id2label.keys())
    
    # ==================== PER-CLASS EVALUATION ====================
    print("\n" + "="*60)
    print("DETAILED PER-CLASS EVALUATION METRICS")
    print("="*60)
    
    # Convert to string labels only for display
    all_pred_labels = [id2label[p] for p in all_preds]
    all_true_labels = [id2label[t] for t in all_targets]
    
    report = classification_report(
        all_true_labels, 
        all_pred_labels, 
        target_names=classes,
        digits=4,
        output_dict=True
    )
    
    class_metrics = pd.DataFrame(report).transpose().drop(['accuracy', 'macro avg', 'weighted avg'])
    print("\nPer-class metrics:")
    print(class_metrics.to_markdown(tablefmt="grid", floatfmt=".4f"))
    
    # ==================== OVERALL EVALUATION ====================
    print("\n" + "="*60)
    print("COMPREHENSIVE OVERALL EVALUATION METRICS")
    print("="*60)
    
    # Use numeric labels (all_targets and all_preds) to calculate metrics
    accuracy = accuracy_score(all_targets, all_preds)
    balanced_accuracy = balanced_accuracy_score(all_targets, all_preds)
    f1_macro = f1_score(all_targets, all_preds, average='macro')
    f1_weighted = f1_score(all_targets, all_preds, average='weighted')
    kappa = cohen_kappa_score(all_targets, all_preds)
    mcc = matthews_corrcoef(all_targets, all_preds)
    lloss = log_loss(all_targets, all_probs, labels=class_ids)
    h_loss = hamming_loss(all_targets, all_preds)
    jaccard = jaccard_score(all_targets, all_preds, average='weighted')
    
    try:
        top2_acc = top_k_accuracy_score(all_targets, all_probs, k=2)
        top3_acc = top_k_accuracy_score(all_targets, all_probs, k=3)
    except:
        top2_acc = top3_acc = None
    
    # ROC AUC calculation
    try:
        if len(classes) == 2:
            roc_auc = roc_auc_score(all_targets, [p[1] for p in all_probs])
        else:
            y_true_bin = label_binarize(all_targets, classes=class_ids)
            roc_auc = roc_auc_score(y_true_bin, all_probs, multi_class='ovr')
    except Exception as e:
        print(f"Could not calculate ROC AUC: {str(e)}")
        roc_auc = None
    
    # Precision-Recall AUC
    try:
        if len(classes) == 2:
            precision, recall, _ = precision_recall_curve(all_targets, [p[1] for p in all_probs])
            pr_auc = auc(recall, precision)
        else:
            pr_auc = average_precision_score(
                label_binarize(all_targets, classes=class_ids),
                all_probs,
                average='macro'
            )
    except Exception as e:
        print(f"Could not calculate PR AUC: {str(e)}")
        pr_auc = None
    
    # Display overall metrics in a comprehensive table
    overall_metrics = {
        'Accuracy': accuracy,
        'Balanced Accuracy': balanced_accuracy,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Cohen Kappa': kappa,
        'Matthews Corr Coef': mcc,
        'Log Loss': lloss,
        'Hamming Loss': h_loss,
        'Jaccard Score': jaccard,
    }
    
    if roc_auc is not None:
        overall_metrics['ROC AUC'] = roc_auc
    if pr_auc is not None:
        overall_metrics['PR AUC'] = pr_auc
    if top2_acc is not None:
        overall_metrics['Top-2 Accuracy'] = top2_acc
        overall_metrics['Top-3 Accuracy'] = top3_acc
    
    overall_df = pd.DataFrame.from_dict(overall_metrics, orient='index', columns=['Value'])
    print("\nOverall metrics:")
    print(overall_df.to_markdown(tablefmt="grid", floatfmt=".4f"))
    
    # ==================== VISUALIZATIONS ====================
    print("\n" + "="*60)
    print("EVALUATION VISUALIZATIONS")
    print("="*60)
    
    # 1. Confusion matrix
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(all_true_labels, all_pred_labels, labels=classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    confusion_matrix_path = os.path.join(log_dir, 'confusion_matrix.png')
    plt.savefig(confusion_matrix_path)
    plt.close()
    print(f"\nConfusion matrix saved to {confusion_matrix_path}")
    
    # 2. Precision-Recall Curve
    plt.figure(figsize=(12, 10))
    if len(classes) == 2:
        precision, recall, _ = precision_recall_curve(all_targets, [p[1] for p in all_probs])
        plt.plot(recall, precision, lw=2, label=f'PR Curve (AUC = {pr_auc:.2f})')
    else:
        for i, class_name in enumerate(classes):
            precision, recall, _ = precision_recall_curve(
                (np.array(all_targets) == class_ids[i]).astype(int),
                np.array(all_probs)[:, i]
            )
            plt.plot(recall, precision, lw=2, label=f'{class_name}')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='best')
    pr_curve_path = os.path.join(log_dir, 'precision_recall_curve.png')
    plt.savefig(pr_curve_path)
    plt.close()
    print(f"Precision-Recall curve saved to {pr_curve_path}")
    
    # 3. ROC Curve (for binary or multiclass)
    if roc_auc is not None:
        plt.figure(figsize=(12, 10))
        if len(classes) == 2:
            fpr, tpr, _ = roc_curve(all_targets, [p[1] for p in all_probs])
            plt.plot(fpr, tpr, lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
        else:
            y_true_bin = label_binarize(all_targets, classes=class_ids)
            for i, class_name in enumerate(classes):
                fpr, tpr, _ = roc_curve(y_true_bin[:, i], np.array(all_probs)[:, i])
                plt.plot(fpr, tpr, lw=2, label=f'{class_name}')
        
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend(loc='best')
        roc_curve_path = os.path.join(log_dir, 'roc_curve.png')
        plt.savefig(roc_curve_path)
        plt.close()
        print(f"ROC curve saved to {roc_curve_path}")
    
    # Log all metrics
    logging.info("\n=== DETAILED PER-CLASS EVALUATION METRICS ===")
    logging.info("\nPer-class metrics:\n" + class_metrics.to_markdown(tablefmt="grid", floatfmt=".4f"))
    logging.info("\n=== COMPREHENSIVE OVERALL EVALUATION METRICS ===")
    logging.info("\nOverall metrics:\n" + overall_df.to_markdown(tablefmt="grid", floatfmt=".4f"))
    
    return {
        'class_metrics': class_metrics,
        'overall_metrics': overall_metrics,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'visualizations': {
            'confusion_matrix': confusion_matrix_path,
            'pr_curve': pr_curve_path,
            'roc_curve': roc_curve_path if roc_auc is not None else None
        }
    }

# Evaluation and Visualization

In [12]:
# Train and Evaluate with Cross-Validation
def train_and_evaluate(df, test_df, tokenizer):
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=86)
    best_accuracy = 0
    best_model_path = None
    
    for fold, (train_index, val_index) in enumerate(skf.split(df, df['sentiment'])):
        print(f'\nFold {fold + 1}/{N_SPLITS}')
        train_df = df.iloc[train_index].reset_index(drop=True)
        val_df = df.iloc[val_index].reset_index(drop=True)
        train_loader, val_loader, _ = prepare_loaders(train_df, val_df, test_df, tokenizer)
        
        model = SentimentClassifier().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = AdamW(model.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * EPOCHS)
        
        for epoch in range(EPOCHS):
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
            val_loss, val_acc = eval_model(model, val_loader, criterion, device)
            logging.info(f"Fold {fold+1}, Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            print(f"Epoch {epoch+1}/{EPOCHS} - Val Acc: {val_acc:.4f}")
            
            if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_model_path = os.path.join(log_dir, f'PhoBERT_sentiment_temp.pth')
                torch.save(model.state_dict(), best_model_path)
                logging.info(f"Saved best model at fold {fold+1}, epoch {epoch+1} with accuracy {val_acc:.4f}")
    
    model.load_state_dict(torch.load(best_model_path))
    os.remove(best_model_path)
    final_model_path = os.path.join(log_dir, 'PhoBERT_summary_sentiment_v2.pth')
    torch.save(model.state_dict(), final_model_path)
    logging.info(f"Final best model saved as PhoBERT_summary_sentiment_v2.pth with accuracy {best_accuracy:.4f}")
    return model

In [13]:
# Inference Function
def predict_sentiment(text, model, tokenizer, id2label):
    dataset = SentimentDataset(pd.DataFrame({'text': [text], 'sentiment': [0]}), tokenizer)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            _, pred = torch.max(outputs, dim=1)
    return id2label[pred.item()]

In [14]:
# Zip output files for download
def zip_and_download_output_files():
    output_zip = os.path.join(log_dir, 'output_files.zip')
    output_files = [
        'training_summary_sentiment_log_v2.txt',
        'data_processed_for_summary_sentiment.csv',
        'PhoBERT_summary_sentiment_v2.pth',
        'confusion_matrix.png',
        'precision_recall_curve.png',
        'roc_curve.png'
    ]
    
    # Only include existing files
    existing_files = [f for f in output_files if os.path.exists(os.path.join(log_dir, f))]
    missing_files = set(output_files) - set(existing_files)
    
    if missing_files:
        logging.warning(f"Missing files: {missing_files}")
    
    try:
        with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file in existing_files:
                file_path = os.path.join(log_dir, file)
                zipf.write(file_path, file)
                logging.info(f"Added {file} to zip archive")
        
        if os.path.exists(output_zip):
            logging.info(f"Zip file created at {output_zip}")
            
            # Automatic download
            print("\n" + "="*60)
            print("AUTOMATICALLY DOWNLOADING OUTPUT FILES")
            print("="*60)
            
            # For Kaggle
            if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
                print("In Kaggle environment, please download manually:")
                display(FileLink('output_files.zip'))
            # For Google Colab
            else:
                print("Downloading output files automatically...")
                from google.colab import files
                files.download(output_zip)
            
            return output_zip
        else:
            logging.error("Failed to create zip file")
            return None
    except Exception as e:
        logging.error(f"Error creating zip: {e}")
        return None

# Main Execution

In [15]:
# Main Execution
if __name__ == "__main__":
    try:
        # Load and preprocess data
        xlsx_path = '/kaggle/input/data-news-v3/data_news_v3.xlsx'
        if not os.path.exists(xlsx_path):
            raise FileNotFoundError(f"File {xlsx_path} not found!")
        print(f"Found data file at {xlsx_path}")
        print("Loading data...")
        df_processed = load_original_data(xlsx_path)
        print("Cleaning data...")
        df_processed = clean_data(df_processed)
        print("Processing labels...")
        df_processed, label2id, id2label = process_labels(df_processed)
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        print("Tokenizer loaded successfully.")
        print("Tokenizing data...")
        df_processed = tokenize_data(df_processed, tokenizer)

        # Save Processed Data
        processed_data_path = os.path.join(log_dir, 'data_processed_for_summary_sentiment.csv')
        df_processed.to_csv(processed_data_path, index=False)
        logging.info(f"Saved processed data with shape: {df_processed.shape}")

        # Statistics and Log
        summary_lengths = df_processed['text'].str.len()
        token_lengths = [len(t) for t in df_processed['tokenized']]
        logging.info(f"Summary length stats: Min={summary_lengths.min()}, Max={summary_lengths.max()}, Mean={summary_lengths.mean():.2f}")
        logging.info(f"Token length stats: Min={min(token_lengths)}, Max={max(token_lengths)}, Mean={np.mean(token_lengths):.2f}")
        logging.info(f"Processing config: max_len={MAX_LEN}, epochs={EPOCHS}, batch_size={BATCH_SIZE}")

        # Split into train+val and test
        train_val_df, test_df = train_test_split(df_processed, test_size=0.2, stratify=df_processed['sentiment'], random_state=86)
        logging.info(f"Data split: Train+Val={len(train_val_df)}, Test={len(test_df)}")

        # Train and evaluate
        print("Starting training...")
        model = train_and_evaluate(train_val_df, test_df, tokenizer)

        # Evaluate on test set
        print("Evaluating model...")
        _, _, test_loader = prepare_loaders(train_val_df, train_val_df, test_df, tokenizer)
        evaluation_results = evaluate_model(model, test_loader, id2label)

        # Test inference
        sample_text = "Đầu tư nước ngoài, dù đã tăng trưởng (FDI đạt 8,9 tỷ USD trong 5 tháng đầu năm), nhưng vẫn ghi nhận dòng vốn ngoại rút mạnh do lo ngại rủi ro thương mại, dù được kỳ vọng sẽ phục hồi khi các thị trường nâng hạng ."
        predicted_sentiment = predict_sentiment(sample_text, model, tokenizer, id2label)
        print(f"\nSample text: {sample_text}")
        print(f"Predicted sentiment: {predicted_sentiment}")
        logging.info(f"Inference test: Text='{sample_text}', Predicted='{predicted_sentiment}'")

        # Zip and automatically download output files
        print("\nPreparing output files for download...")
        zip_and_download_output_files()

    except Exception as e:
        logging.error(f"Error in main execution: {e}", exc_info=True)
        raise

Found data file at /kaggle/input/data-news-v3/data_news_v3.xlsx
Loading data...
Cleaning data...
Processing labels...
Loading tokenizer...


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Epoch 1/10 - Val Acc: 0.7347
Epoch 2/10 - Val Acc: 0.7585
Epoch 3/10 - Val Acc: 0.7565
Epoch 4/10 - Val Acc: 0.7524
Epoch 5/10 - Val Acc: 0.7626
Epoch 6/10 - Val Acc: 0.7497
Epoch 7/10 - Val Acc: 0.7565
Epoch 8/10 - Val Acc: 0.7469
Epoch 9/10 - Val Acc: 0.7551
Epoch 10/10 - Val Acc: 0.7572

Fold 2/5
Epoch 1/10 - Val Acc: 0.7442
Epoch 2/10 - Val Acc: 0.7517
Epoch 3/10 - Val Acc: 0.7517
Epoch 4/10 - Val Acc: 0.7613
Epoch 5/10 - Val Acc: 0.7449
Epoch 6/10 - Val Acc: 0.7551
Epoch 7/10 - Val Acc: 0.7517
Epoch 8/10 - Val Acc: 0.7517
Epoch 9/10 - Val Acc: 0.7510
Epoch 10/10 - Val Acc: 0.7544

Fold 3/5
Epoch 1/10 - Val Acc: 0.7399
Epoch 2/10 - Val Acc: 0.7276
Epoch 3/10 - Val Acc: 0.7536
Epoch 4/10 - Val Acc: 0.7515
Epoch 5/10 - Val Acc: 0.7631
Epoch 6/10 - Val Acc: 0.7495
Epoch 7/10 - Val Acc: 0.7468
Epoch 8/10 - Val Acc: 0.7611
Epoch 9/10 - Val Acc: 0.7631
Epoch 10/10 - Val Acc: 0.7597

Fold 4/5
Epoch 1/10 - Val Acc: 0.7590
Epoch 2/10 - Val Acc: 0.7631
Epoch 3/10 - Val Acc: 0.7625
Epoch 4/10

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1802: UndefinedMetricWarning: 'k' (3) greater than or equal to 'n_classes' (3) will result in a perfect score and is therefore meaningless.
  warnings.warn(



Confusion matrix saved to /kaggle/working/confusion_matrix.png
Precision-Recall curve saved to /kaggle/working/precision_recall_curve.png
ROC curve saved to /kaggle/working/roc_curve.png

Sample text: Đầu tư nước ngoài, dù đã tăng trưởng (FDI đạt 8,9 tỷ USD trong 5 tháng đầu năm), nhưng vẫn ghi nhận dòng vốn ngoại rút mạnh do lo ngại rủi ro thương mại, dù được kỳ vọng sẽ phục hồi khi các thị trường nâng hạng .
Predicted sentiment: Neutral

Preparing output files for download...

AUTOMATICALLY DOWNLOADING OUTPUT FILES
In Kaggle environment, please download manually:


/kaggle/working/output_files.zip